In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ============================================================
# FIXED PREPROCESSING: match Image_###.tif with Label_###.tif
# ============================================================

import os
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

DATA_DIR = Path("/kaggle/input/private-data-1/patches")
IMG_DIR = DATA_DIR / "Images"
LBL_DIR = DATA_DIR / "Labels"

OUT_DIR = Path("/kaggle/working/preprocessed_data")
(OUT_DIR / "images").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "pseudo_labels").mkdir(exist_ok=True)
(OUT_DIR / "importance_maps").mkdir(exist_ok=True)
(OUT_DIR / "similarity_maps").mkdir(exist_ok=True)

IMG_SIZE = 512

def normalize_image(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img.astype(np.float32) / 255.0

def normalize_label(label):
    label = cv2.resize(label, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    label = label.astype(np.float32)
    if label.ndim == 3:
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
    label /= label.max() if label.max() != 0 else 1.0
    return label

def compute_importance_map(label):
    weights = label / (np.sum(label) + 1e-6)
    return weights

def compute_similarity_map(img):
    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    edge_strength = np.sqrt(grad_x**2 + grad_y**2)
    similarity_map = np.exp(-edge_strength / (np.max(edge_strength) + 1e-6))
    return similarity_map

# --- MAIN LOOP ---
image_files = sorted(list(IMG_DIR.glob("*.tif")))
label_files = sorted(list(LBL_DIR.glob("*.tif")))

print(f"Found {len(image_files)} image tiles and {len(label_files)} label tiles...")
processed = 0

for img_path in tqdm(image_files, desc="Preprocessing"):
    # Extract numeric ID from image name
    img_id = ''.join([ch for ch in img_path.stem if ch.isdigit()])
    if not img_id:
        continue

    lbl_path = LBL_DIR / f"Label_{img_id}.tif"
    if not lbl_path.exists():
        continue

    img = cv2.imread(str(img_path))
    lbl = cv2.imread(str(lbl_path), cv2.IMREAD_GRAYSCALE)
    if img is None or lbl is None:
        continue

    img_norm = normalize_image(img)
    lbl_norm = normalize_label(lbl)
    imp_map = compute_importance_map(lbl_norm)
    sim_map = compute_similarity_map(img_norm)

    np.save(OUT_DIR / "images" / f"tile_{img_id}.npy", img_norm)
    np.save(OUT_DIR / "pseudo_labels" / f"tile_{img_id}.npy", lbl_norm)
    np.save(OUT_DIR / "importance_maps" / f"tile_{img_id}.npy", imp_map)
    np.save(OUT_DIR / "similarity_maps" / f"tile_{img_id}.npy", sim_map)

    processed += 1

print(f"✅ Preprocessing complete. Saved {processed} image-label pairs.")


Found 2400 image tiles and 2400 label tiles...


Preprocessing: 100%|██████████| 2400/2400 [01:38<00:00, 24.25it/s]

✅ Preprocessing complete. Saved 2400 image-label pairs.


In [3]:
# ============================================================
# PyTorch Dataset for Weakly Supervised Segmentation
# ------------------------------------------------------------
# Loads preprocessed data:
#   images/
#   pseudo_labels/
#   importance_maps/
#   similarity_maps/
# ------------------------------------------------------------
# Author: thxshsv (Anirudh International LLC)
# ============================================================

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import random

# ============================================================
# CONFIG
# ============================================================
DATA_DIR = Path("/kaggle/working/preprocessed_data")

# ============================================================
# DATASET CLASS
# ============================================================
class PotsdamWeaklyDataset(Dataset):
    def __init__(self, data_dir, transform=None, augment=False):
        """
        Args:
            data_dir (Path): path to the directory containing subfolders:
                             images/, pseudo_labels/, importance_maps/, similarity_maps/
            transform: optional torchvision transform
            augment (bool): whether to apply random augmentations
        """
        self.data_dir = Path(data_dir)
        self.img_paths = sorted((self.data_dir / "images").glob("*.npy"))
        self.transform = transform
        self.augment = augment

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        name = img_path.stem

        img = np.load(img_path)
        label = np.load(self.data_dir / "pseudo_labels" / f"{name}.npy")
        imp = np.load(self.data_dir / "importance_maps" / f"{name}.npy")
        sim = np.load(self.data_dir / "similarity_maps" / f"{name}.npy")

        # --- Normalize and convert to tensors ---
        img = torch.from_numpy(img.transpose(2, 0, 1)).float()  # (C,H,W)
        label = torch.from_numpy(label).unsqueeze(0).float()     # (1,H,W)
        imp = torch.from_numpy(imp).unsqueeze(0).float()         # (1,H,W)
        sim = torch.from_numpy(sim).unsqueeze(0).float()         # (1,H,W)

        # --- Optional Data Augmentation ---
        if self.augment:
            if random.random() > 0.5:
                img = torch.flip(img, dims=[2])
                label = torch.flip(label, dims=[2])
                imp = torch.flip(imp, dims=[2])
                sim = torch.flip(sim, dims=[2])
            if random.random() > 0.5:
                img = torch.flip(img, dims=[1])
                label = torch.flip(label, dims=[1])
                imp = torch.flip(imp, dims=[1])
                sim = torch.flip(sim, dims=[1])

        # --- Apply transform if any ---
        if self.transform:
            img = self.transform(img)

        return {
            "image": img,
            "label": label,
            "importance": imp,
            "similarity": sim,
            "name": name
        }

# ============================================================
# DATALOADER CREATION
# ============================================================
def get_dataloaders(batch_size=4, num_workers=2, augment=True):
    dataset = PotsdamWeaklyDataset(DATA_DIR, augment=augment)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=True)
    return train_loader, val_loader


In [4]:
# # ============================================================
# # WEAKLY-SUPERVISED SEGMENTATION TRAINING (KAGGLE)
# # ============================================================
# # Author: thxshsv (Anirudh International LLC)
# # Dataset: ISPRS Potsdam patched
# # Input:  /kaggle/working/preprocessed_data/
# # Output: /kaggle/working/models/unet_model.pth
# # ============================================================

# !pip install segmentation-models-pytorch --quiet
# import torch
# import torch.nn.functional as F
# from torch import nn, optim
# from tqdm import tqdm
# import segmentation_models_pytorch as smp
# from pathlib import Path
# import numpy as np

# # ============================================================
# # 1. CONFIG
# # ============================================================
# DATA_DIR = Path("/kaggle/working/preprocessed_data")
# MODEL_DIR = Path("/kaggle/working/models")
# MODEL_DIR.mkdir(parents=True, exist_ok=True)

# BATCH_SIZE = 8
# EPOCHS = 20
# LR = 1e-4
# LAMBDA_IMP = 0.5
# LAMBDA_SIM = 0.2

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Running on:", device)

# # ============================================================
# # 2. DATASET + DATALOADER (from previous step)
# # ============================================================
# from torch.utils.data import Dataset, DataLoader
# import random

# class PotsdamWeaklyDataset(Dataset):
#     def __init__(self, data_dir, augment=False):
#         self.data_dir = Path(data_dir)
#         self.img_paths = sorted((self.data_dir / "images").glob("*.npy"))
#         self.augment = augment

#     def __len__(self):
#         return len(self.img_paths)

#     def __getitem__(self, idx):
#         name = self.img_paths[idx].stem
#         img = np.load(self.data_dir / "images" / f"{name}.npy")
#         lbl = np.load(self.data_dir / "pseudo_labels" / f"{name}.npy")
#         imp = np.load(self.data_dir / "importance_maps" / f"{name}.npy")
#         sim = np.load(self.data_dir / "similarity_maps" / f"{name}.npy")

#         img = torch.from_numpy(img.transpose(2, 0, 1)).float()
#         lbl = torch.from_numpy(lbl).unsqueeze(0).float()
#         imp = torch.from_numpy(imp).unsqueeze(0).float()
#         sim = torch.from_numpy(sim).unsqueeze(0).float()

#         if self.augment:
#             if random.random() > 0.5:
#                 img = torch.flip(img, dims=[2])
#                 lbl = torch.flip(lbl, dims=[2])
#                 imp = torch.flip(imp, dims=[2])
#                 sim = torch.flip(sim, dims=[2])
#             if random.random() > 0.5:
#                 img = torch.flip(img, dims=[1])
#                 lbl = torch.flip(lbl, dims=[1])
#                 imp = torch.flip(imp, dims=[1])
#                 sim = torch.flip(sim, dims=[1])

#         return {"image": img, "label": lbl, "importance": imp, "similarity": sim, "name": name}

# def get_dataloaders(batch_size=8, augment=True):
#     dataset = PotsdamWeaklyDataset(DATA_DIR, augment=augment)
#     n = len(dataset)
#     train_size = int(0.8 * n)
#     val_size = n - train_size
#     train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])
#     train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
#     val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
#     return train_loader, val_loader

# train_loader, val_loader = get_dataloaders(BATCH_SIZE)

# # ============================================================
# # 3. MODEL
# # ============================================================
# model = smp.Unet(
#     encoder_name="resnet34",
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1
# ).to(device)

# optimizer = optim.Adam(model.parameters(), lr=LR)

# # ============================================================
# # 4. METRICS
# # ============================================================
# def dice_coeff(pred, target, eps=1e-7):
#     pred = (pred > 0.5).float()
#     inter = (pred * target).sum()
#     return (2 * inter + eps) / (pred.sum() + target.sum() + eps)

# def iou_score(pred, target, eps=1e-7):
#     pred = (pred > 0.5).float()
#     inter = (pred * target).sum()
#     union = pred.sum() + target.sum() - inter
#     return (inter + eps) / (union + eps)

# # ============================================================
# # 5. TRAINING LOOP
# # ============================================================
# def compute_similarity_loss(preds_sig, sim):
#     """Spatial smoothness: encourages similar predictions in high-similarity regions."""
#     diff_x = (preds_sig[:, :, 1:, :] - preds_sig[:, :, :-1, :])**2
#     diff_y = (preds_sig[:, :, :, 1:] - preds_sig[:, :, :, :-1])**2
#     sim_x = sim[:, :, 1:, :]
#     sim_y = sim[:, :, :, 1:]
#     loss_x = (sim_x * diff_x).mean()
#     loss_y = (sim_y * diff_y).mean()
#     return loss_x + loss_y

# def train_one_epoch(loader):
#     model.train()
#     total_loss, total_dice = 0, 0
#     for batch in tqdm(loader, desc="Training"):
#         imgs = batch["image"].to(device)
#         labels = batch["label"].to(device)
#         imp = batch["importance"].to(device)
#         sim = batch["similarity"].to(device)

#         preds = model(imgs)
#         preds_sig = torch.sigmoid(preds)

#         # Losses
#         sup_loss = F.binary_cross_entropy(preds_sig, labels)
#         imp_loss = torch.mean(imp * (preds_sig - labels) ** 2)
#         sim_loss = compute_similarity_loss(preds_sig, sim)
#         loss = sup_loss + LAMBDA_IMP * imp_loss + LAMBDA_SIM * sim_loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()
#         total_dice += dice_coeff(preds_sig, labels).item()
#     return total_loss / len(loader), total_dice / len(loader)

# @torch.no_grad()
# def validate_one_epoch(loader):
#     model.eval()
#     total_loss, total_dice, total_iou = 0, 0, 0
#     for batch in tqdm(loader, desc="Validation"):
#         imgs = batch["image"].to(device)
#         labels = batch["label"].to(device)
#         imp = batch["importance"].to(device)
#         sim = batch["similarity"].to(device)

#         preds = model(imgs)
#         preds_sig = torch.sigmoid(preds)

#         sup_loss = F.binary_cross_entropy(preds_sig, labels)
#         imp_loss = torch.mean(imp * (preds_sig - labels) ** 2)
#         sim_loss = compute_similarity_loss(preds_sig, sim)
#         loss = sup_loss + LAMBDA_IMP * imp_loss + LAMBDA_SIM * sim_loss

#         total_loss += loss.item()
#         total_dice += dice_coeff(preds_sig, labels).item()
#         total_iou += iou_score(preds_sig, labels).item()

#     n = len(loader)
#     return total_loss / n, total_dice / n, total_iou / n

# # ============================================================
# # 6. TRAINING
# # ============================================================
# best_dice = 0
# for epoch in range(EPOCHS):
#     train_loss, train_dice = train_one_epoch(train_loader)
#     val_loss, val_dice, val_iou = validate_one_epoch(val_loader)

#     print(f"\nEpoch {epoch+1}/{EPOCHS}")
#     print(f"Train Loss: {train_loss:.4f} | Train Dice: {train_dice:.4f}")
#     print(f"Val   Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f} | Val IoU: {val_iou:.4f}")

#     # Save best model
#     if val_dice > best_dice:
#         best_dice = val_dice
#         torch.save(model.state_dict(), MODEL_DIR / "unet_model.pth")
#         print("✅ Model saved (best so far)")

# print("🎯 Training complete. Best Dice:", best_dice)


In [5]:
# ============================================================
# MAVI-Hybrid: CNN + ViT + Attention Fusion for Weak Supervision
# ============================================================
# Dataset: ISPRS Potsdam (weakly-supervised)
# Author: thxshsv (Anirudh International LLC)
# ============================================================

!pip install segmentation-models-pytorch timm --quiet

import torch, torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder
from timm.models.vision_transformer import vit_base_patch16_224
from pathlib import Path
import numpy as np, cv2, random
from tqdm import tqdm

# ============================================================
# CONFIG
# ============================================================
DATA_DIR = Path("/kaggle/working/preprocessed_data")
UNLABELED_IMG_DIR = Path("/kaggle/input/private-data-1/patches/Images")
MODEL_DIR = Path("/kaggle/working/models"); MODEL_DIR.mkdir(parents=True, exist_ok=True)

BATCH_SIZE   = 8
EPOCHS       = 50
LR           = 5e-5
WARMUP_EPOCHS = 3
LAMBDA_IMP   = 0.5
LAMBDA_SIM   = 0.2
REFINE_EVERY = 5
PATIENCE     = 7
UNFREEZE_EPOCH = 5  # unfreeze ViT after 5 epochs

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

# ============================================================
# DATASET
# ============================================================
class PotsdamWeaklyDataset(Dataset):
    def __init__(self, data_dir, augment=False):
        self.data_dir = Path(data_dir)
        self.img_paths = sorted((self.data_dir / "images").glob("*.npy"))
        self.augment = augment
    def __len__(self): return len(self.img_paths)
    def __getitem__(self, idx):
        name = self.img_paths[idx].stem
        img = np.load(self.data_dir / "images" / f"{name}.npy")
        lbl = np.load(self.data_dir / "pseudo_labels" / f"{name}.npy")
        imp = np.load(self.data_dir / "importance_maps" / f"{name}.npy")
        sim = np.load(self.data_dir / "similarity_maps" / f"{name}.npy")

        img = torch.from_numpy(img.transpose(2,0,1)).float()
        lbl = torch.from_numpy(lbl).unsqueeze(0).float()
        imp = torch.from_numpy(imp).unsqueeze(0).float()
        sim = torch.from_numpy(sim).unsqueeze(0).float()

        if self.augment:
            if random.random() > 0.5:
                img,lbl,imp,sim = [torch.flip(t,dims=[2]) for t in [img,lbl,imp,sim]]
            if random.random() > 0.5:
                img,lbl,imp,sim = [torch.flip(t,dims=[1]) for t in [img,lbl,imp,sim]]

        return {"image": img, "label": lbl, "importance": imp, "similarity": sim}

def get_dataloaders(batch_size=BATCH_SIZE, augment=True):
    ds = PotsdamWeaklyDataset(DATA_DIR, augment)
    if len(ds)==0: raise ValueError("No preprocessed .npy found")
    tr_sz = int(0.8*len(ds)); val_sz = len(ds)-tr_sz
    tr,val = torch.utils.data.random_split(ds,[tr_sz,val_sz])
    return (DataLoader(tr,batch_size,True,num_workers=2),
            DataLoader(val,batch_size,False,num_workers=2))

train_loader,val_loader=get_dataloaders()

# ============================================================
# ATTENTION FUSION BLOCK
# ============================================================
class AttentionFusionBlock(nn.Module):
    def __init__(self, cnn_channels=512, vit_dim=768):
        super().__init__()
        self.query = nn.Conv2d(cnn_channels, vit_dim, 1)
        self.key   = nn.Linear(vit_dim, vit_dim)
        self.value = nn.Linear(vit_dim, vit_dim)
        self.proj  = nn.Conv2d(vit_dim, cnn_channels, 1)
        self.scale = vit_dim ** -0.5
    def forward(self, cnn_feats, vit_tokens):
        B,C,H,W = cnn_feats.shape
        q = self.query(cnn_feats).flatten(2).transpose(1,2)
        k = self.key(vit_tokens); v = self.value(vit_tokens)
        attn = torch.softmax((q @ k.transpose(-2,-1))*self.scale, dim=-1)
        fused = (attn @ v).transpose(1,2).reshape(B,-1,H,W)
        return self.proj(fused) + cnn_feats

# ============================================================
# MAVI-HYBRID MODEL (Fixed SMP Compatibility)
# ============================================================
# --- keep your existing imports ---
# import segmentation_models_pytorch as smp
# from timm.models.vision_transformer import vit_base_patch16_224
# (and your AttentionFusionBlock)

class MAVIHybridNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Build a full SMP Unet (it creates encoder+decoder wired correctly)
        self.base_unet = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=3,
            classes=1
        )
        # ViT for global tokens
        self.vit = vit_base_patch16_224(pretrained=True)
        # Our fusion acts on the deepest encoder feature
        # ResNet34 deepest feature is 512 ch, ViT-B/16 has 768-d tokens
        self.fusion = AttentionFusionBlock(cnn_channels=512, vit_dim=768)

    def forward(self, x):
        # 1) Encoder features from SMP (list of 5 tensors, shallow -> deep)
        feats = self.base_unet.encoder(x)   # [f0(64), f1(64), f2(128), f3(256), f4(512)]

        # 2) ViT tokens (use resized input for ViT)
        vit_tokens = self.vit.forward_features(F.interpolate(x, size=(224, 224)))

        # 3) Fuse on the deepest feature (keep same 512 ch, same spatial size)
        fused_deep = self.fusion(feats[-1], vit_tokens)

        # 4) Replace deepest feature with fused one
        feats[-1] = fused_deep

        # 5) Decode using SMP’s own decoder (guaranteed channel correctness)
        dec = self.base_unet.decoder(feats)

        # 6) Final segmentation head
        out = self.base_unet.segmentation_head(dec)
        return out

# ============================================================
# MODEL INIT + OPTIMIZER + SCHEDULER
# ============================================================
model = MAVIHybridNet().to(device)
print(f"✅ MAVIHybridNet initialized on {device.upper()}")

# Freeze ViT for stability
for p in model.vit.parameters():
    p.requires_grad = False
print("🔒 ViT encoder frozen for warm-up phase.")

optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print("✅ Optimizer and scheduler initialized.")

# ============================================================
# WARMUP LR
# ============================================================
def warmup_lr(optimizer, epoch, warmup_epochs=WARMUP_EPOCHS, base_lr=LR):
    if epoch < warmup_epochs:
        lr_scale = (epoch + 1) / warmup_epochs
        for pg in optimizer.param_groups:
            pg["lr"] = base_lr * lr_scale
        print(f"🔺 Warmup LR adjusted to {base_lr * lr_scale:.2e}")

# ============================================================
# METRICS / LOSSES
# ============================================================
def dice(pred,gt,eps=1e-7):
    p=(pred>0.5).float(); inter=(p*gt).sum()
    return (2*inter+eps)/(p.sum()+gt.sum()+eps)
def iou(pred,gt,eps=1e-7):
    p=(pred>0.5).float(); inter=(p*gt).sum()
    return (inter+eps)/(p.sum()+gt.sum()-inter+eps)
def sim_loss(pred,sim):
    dx=(pred[:,:,1:,:]-pred[:,:,:-1,:])**2
    dy=(pred[:,:,:,1:]-pred[:,:,:,:-1])**2
    return (sim[:,:,1:,:]*dx).mean()+(sim[:,:,:,1:]*dy).mean()

# ============================================================
# PSEUDO-LABEL REFINEMENT
# ============================================================
def generate_pseudo_labels(model, img_dir=UNLABELED_IMG_DIR, save_dir=DATA_DIR/"pseudo_labels"):
    model.eval(); imgs=sorted(img_dir.glob("*.tif"))
    print(f"🌀 Regenerating pseudo-labels for {len(imgs)} images...")
    for p in tqdm(imgs):
        name=p.stem; img=cv2.imread(str(p))
        if img is None: continue
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB); img=cv2.resize(img,(512,512))
        t=torch.from_numpy(img.transpose(2,0,1)/255.).unsqueeze(0).float().to(device)
        with torch.no_grad(): pred=torch.sigmoid(model(t)).cpu().squeeze().numpy()
        pred=cv2.medianBlur((pred*255).astype(np.uint8),3).astype(np.float32)/255.
        np.save(save_dir/f"{name}.npy",pred)
    print("✅ Pseudo-label update done!")

# ============================================================
# TRAIN / VALIDATE
# ============================================================
def run_epoch(loader,train=True):
    model.train() if train else model.eval()
    totL=totD=totI=0
    for b in tqdm(loader,desc="Train" if train else "Val"):
        x=b["image"].to(device); y=b["label"].to(device)
        imp=b["importance"].to(device); sim=b["similarity"].to(device)
        with torch.set_grad_enabled(train):
            p=torch.sigmoid(model(x))
            sup=F.binary_cross_entropy(p,y)
            impL=(imp*(p-y)**2).mean(); sL=sim_loss(p,sim)
            loss=sup+LAMBDA_IMP*impL+LAMBDA_SIM*sL
            if train: optimizer.zero_grad(); loss.backward(); optimizer.step()
        totL+=loss.item(); totD+=dice(p,y).item(); totI+=iou(p,y).item()
    n=len(loader)
    return totL/n,totD/n,totI/n

# ============================================================
# MAIN LOOP
# ============================================================
best_dice,epochs_no_improve=0,0
for ep in range(EPOCHS):
    print(f"\n========== Epoch {ep+1}/{EPOCHS} ==========")
    warmup_lr(optimizer, ep)

    # Gradual unfreezing
    if ep == UNFREEZE_EPOCH:
        print("🔓 Unfreezing ViT encoder for fine-tuning...")
        for p in model.vit.parameters(): p.requires_grad = True

    tr_l,tr_d,_=run_epoch(train_loader,True)
    val_l,val_d,val_i=run_epoch(val_loader,False)
    scheduler.step()
    print(f"Train L:{tr_l:.4f}|D:{tr_d:.4f}  Val L:{val_l:.4f}|D:{val_d:.4f}|IoU:{val_i:.4f}")

    # Save best model
    if val_d>best_dice:
        best_dice=val_d; epochs_no_improve=0
        torch.save(model.state_dict(), MODEL_DIR/"mavi_hybrid_best.pth")
        print("✅ Saved best model.")
    else:
        epochs_no_improve+=1
        if epochs_no_improve>=PATIENCE:
            print("⏹ Early stopping triggered."); break

    # Pseudo-label refinement
    if (ep+1)%REFINE_EVERY==0:
        print(f"🔁 Refining pseudo-labels after epoch {ep+1}")
        generate_pseudo_labels(model)
        train_loader,val_loader=get_dataloaders()

print(f"🎯 Training complete. Best Dice={best_dice:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Running on: cuda


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

✅ MAVIHybridNet initialized on CUDA
🔒 ViT encoder frozen for warm-up phase.
✅ Optimizer and scheduler initialized.

========== Epoch 1/50 ==========
🔺 Warmup LR adjusted to 1.67e-05


Train:   0%|          | 0/240 [00:00<?, ?it/s]

In [ ]:
# ==============================================
# EVALUATION (robust saving + PR/AP + threshold sweep)
# ==============================================
import numpy as np, torch, shutil, cv2, matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, average_precision_score

METRICS_DIR = Path("/kaggle/working/metrics"); METRICS_DIR.mkdir(parents=True, exist_ok=True)
PREDS_DIR   = Path("/kaggle/working/preds")

def _safe_save(path, arr, mode="png"):
    """Save probability or mask with minimal IO issues."""
    try:
        if mode == "png":
            a = np.clip(arr, 0, 1) * 255.0
            a = a.astype(np.uint8)
            return bool(cv2.imwrite(str(path.with_suffix(".png")), a))
        elif mode == "npz":
            np.savez_compressed(path.with_suffix(".npz"), prob=arr.astype(np.float16))
            return True
        else:
            np.save(path.with_suffix(".npy"), arr); return True
    except OSError as e:
        print(f"⚠️ Save failed at {path.name}: {e}")
        return False

def dice_score(y_true_bin, y_pred_bin, eps=1e-7):
    inter = (y_true_bin & y_pred_bin).sum()
    return (2*inter + eps) / (y_true_bin.sum() + y_pred_bin.sum() + eps)

def iou_score(y_true_bin, y_pred_bin, eps=1e-7):
    inter = (y_true_bin & y_pred_bin).sum()
    union = y_true_bin.sum() + y_pred_bin.sum() - inter
    return (inter + eps) / (union + eps)

@torch.no_grad()
def evaluate_model(
    model,
    loader,
    device,
    default_thresh=0.5,
    max_save=120,         # limit disk writes; set 0 to skip saving
    save_mode="png",      # 'png'|'npz'|'npy'
    save_masks=True
):
    # clean preds dir
    if PREDS_DIR.exists():
        shutil.rmtree(PREDS_DIR)
    (PREDS_DIR/"probs").mkdir(parents=True, exist_ok=True)
    (PREDS_DIR/"masks").mkdir(parents=True, exist_ok=True)

    model.eval()
    all_probs = []
    all_gts   = []
    save_idx = 0
    saving_ok = True

    for batch in tqdm(loader, desc="Eval/Val"):
        x = batch["image"].to(device)
        y = batch["label"].cpu().numpy()          # (B,1,H,W) weak labels in [0,1]
        p = torch.sigmoid(model(x)).cpu().numpy()  # (B,1,H,W)

        for i in range(p.shape[0]):
            prob = p[i,0]
            gt   = y[i,0]
            all_probs.append(prob.ravel())
            all_gts.append(gt.ravel())

            if saving_ok and save_idx < max_save:
                ok1 = _safe_save((PREDS_DIR/"probs"/f"val_{save_idx:06d}"), prob, mode=save_mode)
                ok2 = True
                if save_masks:
                    mask = (prob >= default_thresh).astype(np.uint8) * 255
                    ok2 = bool(cv2.imwrite(str((PREDS_DIR/"masks"/f"val_{save_idx:06d}").with_suffix(".png")), mask))
                if not (ok1 and ok2):
                    print("💾 Disk pressure: switching to metrics-only (no further saves).")
                    saving_ok = False
            save_idx += 1

    all_probs = np.concatenate(all_probs, axis=0)
    all_gts   = np.concatenate(all_gts,   axis=0)

    # ---- Binzarize GT for PR/AP ----
    gt_bin_vec = (all_gts >= 0.5).astype(np.uint8)
    m = np.isfinite(all_probs) & np.isfinite(gt_bin_vec)
    all_probs_c = all_probs[m]
    gt_bin_vec_c = gt_bin_vec[m]

    # PR + AP
    precision, recall, pr_thresholds = precision_recall_curve(gt_bin_vec_c, all_probs_c)
    ap = average_precision_score(gt_bin_vec_c, all_probs_c)

    # Threshold sweep (Dice/IoU)
    sweep = np.linspace(0.05, 0.95, 19)
    dice_list, iou_list = [], []
    gt_bool = gt_bin_vec_c.astype(bool)
    for t in sweep:
        pred_bin = (all_probs_c >= t)
        dice_list.append(dice_score(gt_bool, pred_bin))
        iou_list.append(iou_score(gt_bool, pred_bin))

    best_idx = int(np.argmax(dice_list))
    best_t   = float(sweep[best_idx])
    best_d   = float(dice_list[best_idx])
    best_i   = float(iou_list[best_idx])

    # --- plots ---
    plt.figure(figsize=(5,4))
    plt.plot(recall, precision)
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR curve (AP={ap:.4f})")
    plt.grid(True, alpha=0.3); plt.tight_layout()
    plt.savefig(METRICS_DIR/"pr_curve.png", dpi=150); plt.close()

    plt.figure(figsize=(5,4))
    plt.plot(sweep, dice_list, label="Dice")
    plt.plot(sweep, iou_list,  label="IoU", alpha=0.8)
    plt.axvline(best_t, ls="--", color="k", label=f"Best t={best_t:.2f}")
    plt.xlabel("Threshold"); plt.ylabel("Score"); plt.title("Threshold sweep")
    plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout()
    plt.savefig(METRICS_DIR/"dice_iou_vs_threshold.png", dpi=150); plt.close()

    with open(METRICS_DIR/"summary.txt","w") as f:
        f.write(f"Average Precision (AP): {ap:.6f}\n")
        f.write(f"Best threshold (Dice): {best_t:.4f}\n")
        f.write(f"Best Dice: {best_d:.6f}\n")
        f.write(f"Best IoU:  {best_i:.6f}\n")
        f.write(f"Default threshold saved masks: {default_thresh:.2f}\n")
        f.write(f"Files saved (max_save): {min(save_idx, max_save)}; save_mode={save_mode}\n")

    print("============================================")
    print(f"AP: {ap:.4f}")
    print(f"Best t (Dice): {best_t:.3f} | Dice: {best_d:.4f} | IoU: {best_i:.4f}")
    print(f"Saved plots to: {METRICS_DIR}")
    print(f"Saved some predictions to: {PREDS_DIR} (mode={save_mode}, max_save={max_save})")
    print("============================================")

    return {
        "ap": ap, "best_threshold": best_t,
        "best_dice": best_d, "best_iou": best_i,
        "precision": precision, "recall": recall,
        "sweep": sweep, "dice_list": dice_list, "iou_list": iou_list
    }

# ---- Run evaluation ----
eval_results = evaluate_model(
    model, val_loader, device,
    default_thresh=0.5,
    max_save=120,         # reduce if IO errors; set 0 to skip saving
    save_mode="png",      # 'png' is small; 'npz' also good
    save_masks=True
)


In [ ]:
# ==============================================
# VIS: Original Image + Predicted Mask (Overlays)
# ==============================================
import os, cv2, numpy as np, torch
from pathlib import Path
from tqdm import tqdm

OVERLAY_DIR = Path("/kaggle/working/overlays"); OVERLAY_DIR.mkdir(parents=True, exist_ok=True)

# Use best threshold from eval (fallback to 0.55 if not present)
BEST_T = float(eval_results["best_threshold"]) if "eval_results" in globals() else 0.55
print(f"Using threshold = {BEST_T:.2f} for overlays")

def to_numpy_img(t):  # (3,H,W) torch -> (H,W,3) np float [0,1]
    a = t.detach().cpu().numpy()
    a = np.transpose(a, (1,2,0))
    return np.clip(a, 0, 1)

@torch.no_grad()
def save_overlays(model, loader, out_dir=OVERLAY_DIR, max_batches=5):
    model.eval()
    saved = 0
    for b, batch in enumerate(tqdm(loader, desc="Overlays")):
        if b >= max_batches: break
        x = batch["image"].to(device)              # (B,3,H,W)
        probs = torch.sigmoid(model(x)).cpu().numpy()  # (B,1,H,W)

        for i in range(x.size(0)):
            img = (to_numpy_img(x[i]) * 255).astype(np.uint8)      # RGB uint8
            prob = probs[i,0]                                      # (H,W) float
            mask = (prob >= BEST_T).astype(np.uint8)               # (H,W) {0,1}

            # Colorize mask (green) and blend on top of image
            color = np.zeros_like(img)
            color[:,:,1] = (mask * 255)  # put mask in G channel
            overlay = cv2.addWeighted(img, 1.0, color, 0.45, 0)    # 45% mask

            # Also add a probability heatmap side-by-side (optional)
            heat = (prob * 255).astype(np.uint8)
            heat = cv2.applyColorMap(heat, cv2.COLORMAP_TURBO)     # COLORMAP_JET also ok
            heat = cv2.cvtColor(heat, cv2.COLOR_BGR2RGB)

            # Stack: [original | overlay | heatmap]
            grid = np.concatenate([img, overlay, heat], axis=1)
            cv2.imwrite(str(out_dir / f"val_{saved:06d}.png"), cv2.cvtColor(grid, cv2.COLOR_RGB2BGR))
            saved += 1
    print(f"Saved {saved} overlay images to: {out_dir}")

# Run it
save_overlays(model, val_loader, OVERLAY_DIR, max_batches=5)


In [ ]:
# ==============================================
# ROI Compression on RAW .tif with better strategies (metrics-only)
# Baselines: JPEG q=90 and WebP q=80
# Strategies: 
#   - binary_keep_png  (mask bg to 0, save PNG)
#   - soft_keep_webp   (blur+dim bg, WebP q=60)
#   - roi_only_png     (ONLY tight ROI crop as PNG)
#   - roi_crop_ctx     (ROI PNG crop + context WebP q=35)
# ==============================================
import cv2, numpy as np, torch, glob
from pathlib import Path
from tqdm import tqdm

RAW_IMG_DIR = Path("/kaggle/input/private-data-1/patches/Images")
SAVE_FEW_SAMPLES = 0  # set >0 to save previews (uses /kaggle/working, mind disk)
SAMPLES_OUT_DIR = Path("/kaggle/working/roi_outputs_better"); SAMPLES_OUT_DIR.mkdir(exist_ok=True, parents=True)

BEST_T = float(eval_results["best_threshold"]) if "eval_results" in globals() else 0.55
print(f"Using threshold for ROI = {BEST_T:.2f}")

def _enc_jpg(rgb, q): 
    ok, enc = cv2.imencode(".jpg", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, int(q)])
    if not ok: raise RuntimeError("JPEG encode failed"); 
    return enc.tobytes()

def _enc_png(rgb, level=3):
    ok, enc = cv2.imencode(".png", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_PNG_COMPRESSION, int(level)])
    if not ok: raise RuntimeError("PNG encode failed")
    return enc.tobytes()

def _enc_webp(rgb, q=80):
    ok, enc = cv2.imencode(".webp", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_WEBP_QUALITY, int(q)])
    if not ok: raise RuntimeError("WEBP encode failed")
    return enc.tobytes()

def _tight_bbox(mask):
    ys, xs = np.where(mask > 0)
    if len(xs) == 0: return None
    return xs.min(), ys.min(), xs.max(), ys.max()

def compress_strategies(rgb, mask01):
    """Return dict of byte counts for different strategies, plus (optional) preview images."""
    H, W = mask01.shape
    mask3 = np.dstack([mask01]*3).astype(np.uint8)

    # Baselines
    base_jpg90 = len(_enc_jpg(rgb, q=90))
    base_webp80 = len(_enc_webp(rgb, q=80))

    # Strategy 1: binary_keep_png (zero bg) -> PNG
    img_bin = rgb.copy()
    img_bin[mask3 == 0] = 0
    s1 = len(_enc_png(img_bin, level=3))

    # Strategy 2: soft_keep_webp (blur+dim bg) -> WebP q=60
    bg = cv2.GaussianBlur(rgb, (15,15), 0)
    bg = (bg.astype(np.float32) * 0.35).astype(np.uint8)
    soft = (mask3 * rgb + (1 - mask3) * bg).astype(np.uint8)
    s2 = len(_enc_webp(soft, q=60))

    # Strategy 3: roi_only_png (no context)
    bb = _tight_bbox(mask01)
    if bb is None:
        s3 = len(_enc_webp(rgb, q=35))  # no ROI; send a tiny context
    else:
        x1,y1,x2,y2 = bb
        pad = max(4, int(0.01*max(H,W)))
        x1 = max(0, x1 - pad); y1 = max(0, y1 - pad)
        x2 = min(W-1, x2 + pad); y2 = min(H-1, y2 + pad)
        crop = rgb[y1:y2+1, x1:x2+1]
        s3 = len(_enc_png(crop, level=3))

    # Strategy 4: roi_crop_ctx (ROI PNG + tiny context WebP q=35)
    if bb is None:
        s4 = len(_enc_webp(rgb, q=35))
    else:
        crop = rgb[y1:y2+1, x1:x2+1]
        crop_png = len(_enc_png(crop, level=3))
        ctx_webp = len(_enc_webp(rgb, q=35))
        s4 = crop_png + ctx_webp

    sizes = {
        "baseline_jpg90": base_jpg90,
        "baseline_webp80": base_webp80,
        "binary_keep_png": s1,
        "soft_keep_webp": s2,
        "roi_only_png": s3,
        "roi_crop_ctx": s4,
    }
    previews = {
        "bin": img_bin,
        "soft": soft,
        "crop": crop if bb is not None else rgb,
    }
    return sizes, previews
# Minimal diagnostic runner: executes the ROI bench and RETURNS a summary dict
import numpy as np

def run_roi_bench_quick(model, max_images=20, threshold=0.55):
    # Ensure the improved function exists; re-import np inside for safety
    @torch.no_grad()
    def _runner():
        import numpy as np, glob, cv2, torch
        from pathlib import Path
        paths = sorted(glob.glob(str(Path("/kaggle/input/private-data-1/patches/Images") / "*.tif")))
        if len(paths) == 0:
            print("⚠️ No .tif images found"); 
            return None

        agg = {k: [] for k in ["baseline_jpg90","baseline_webp80","binary_keep_png","soft_keep_webp","roi_only_png","roi_crop_ctx"]}
        for p in paths[:max_images]:
            im_bgr = cv2.imread(p)
            if im_bgr is None: 
                continue
            im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
            H, W = im_rgb.shape[:2]

            im_resz = cv2.resize(im_rgb, (512,512))
            t = torch.from_numpy((im_resz/255.0).transpose(2,0,1)).unsqueeze(0).float().to(device)
            prob = torch.sigmoid(model(t)).cpu().numpy()[0,0]
            mask = cv2.resize((prob >= threshold).astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST)

            sizes, _ = compress_strategies(im_rgb, mask)
            for k,v in sizes.items():
                agg[k].append(v)

        if len(agg["baseline_jpg90"]) == 0:
            print("⚠️ No images processed."); 
            return None

        # Build a clean summary dict (averages + % savings)
        b_jpg = np.array(agg["baseline_jpg90"], dtype=np.float64)
        b_wbp = np.array(agg["baseline_webp80"], dtype=np.float64)
        summary = {"vs_jpeg90": {}, "vs_webp80": {}}
        for m in ["binary_keep_png","soft_keep_webp","roi_only_png","roi_crop_ctx"]:
            out = np.array(agg[m], dtype=np.float64)
            summ = {
                "mean_orig_kb": float(b_jpg.mean()/1024),
                "mean_out_kb":  float(out.mean()/1024),
                "saving_pct":   float((1 - out.mean()/b_jpg.mean())*100.0)
            }
            summary["vs_jpeg90"][m] = summ
            summ2 = {
                "mean_orig_kb": float(b_wbp.mean()/1024),
                "mean_out_kb":  float(out.mean()/1024),
                "saving_pct":   float((1 - out.mean()/b_wbp.mean())*100.0)
            }
            summary["vs_webp80"][m] = summ2
        return summary

    res = _runner()
    if res is None:
        print("No results produced.")
    else:
        # pretty print
        print("\n=== Savings vs JPEG q=90 baseline ===")
        for k,v in res["vs_jpeg90"].items():
            print(f"{k:15s} | mean orig: {v['mean_orig_kb']:.1f} KB | mean out: {v['mean_out_kb']:.1f} KB | saving: {v['saving_pct']:.1f}%")
        print("\n=== Savings vs WebP q=80 baseline ===")
        for k,v in res["vs_webp80"].items():
            print(f"{k:15s} | mean orig: {v['mean_orig_kb']:.1f} KB | mean out: {v['mean_out_kb']:.1f} KB | saving: {v['saving_pct']:.1f}%")
    return res

# Call it (uses the same BEST_T from your eval)
_ = run_roi_bench_quick(model, max_images=20, threshold=BEST_T)



In [ ]:
# ==============================================
# Adaptive ROI encoder (metrics-only, space-safe)
# ==============================================
import cv2, numpy as np, torch, glob
from pathlib import Path
from tqdm import tqdm

RAW_IMG_DIR = Path("/kaggle/input/private-data-1/patches/Images")
SAVE_FEW_SAMPLES = 0
SAMPLES_OUT_DIR = Path("/kaggle/working/roi_adaptive_samples"); SAMPLES_OUT_DIR.mkdir(parents=True, exist_ok=True)

BEST_T = float(eval_results["best_threshold"]) if "eval_results" in globals() else 0.55
print(f"Using threshold for ROI = {BEST_T:.2f}")

def _enc_jpg(rgb, q): 
    ok, enc = cv2.imencode(".jpg", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, int(q)])
    return enc.tobytes() if ok else None

def _enc_png(rgb, level=3):
    ok, enc = cv2.imencode(".png", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_PNG_COMPRESSION, int(level)])
    return enc.tobytes() if ok else None

def _enc_webp(rgb, q=80):
    ok, enc = cv2.imencode(".webp", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_WEBP_QUALITY, int(q)])
    return enc.tobytes() if ok else None

def _tight_bbox(mask):
    ys, xs = np.where(mask > 0)
    if len(xs) == 0: return None
    return xs.min(), ys.min(), xs.max(), ys.max()

def _edge_density(gray_uint8):
    gx = cv2.Sobel(gray_uint8, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray_uint8, cv2.CV_32F, 0, 1, ksize=3)
    mag = np.sqrt(gx*gx + gy*gy)
    return float((mag > 25).mean())  # fraction of edge pixels

def _overlay(rgb, mask):
    color = np.zeros_like(rgb); color[:,:,1] = (mask*255)
    return cv2.addWeighted(rgb, 1.0, color, 0.45, 0)

def _binary_keep_png(rgb, mask01):
    m3 = np.dstack([mask01]*3).astype(np.uint8)
    out = rgb.copy(); out[m3==0] = 0
    return _enc_png(out, level=3), out

def _soft_keep_webp(rgb, mask01, q=60):
    m3 = np.dstack([mask01]*3).astype(np.uint8)
    bg = cv2.GaussianBlur(rgb, (15,15), 0)
    bg = (bg.astype(np.float32) * 0.35).astype(np.uint8)
    out = (m3 * rgb + (1 - m3) * bg).astype(np.uint8)
    return _enc_webp(out, q=q), out

def _roi_only_png(rgb, mask01):
    bb = _tight_bbox(mask01)
    if bb is None:
        return _enc_webp(rgb, q=35), rgb
    x1,y1,x2,y2 = bb
    pad = max(4, int(0.01*max(rgb.shape[:2])))
    x1 = max(0, x1 - pad); y1 = max(0, y1 - pad)
    x2 = min(rgb.shape[1]-1, x2 + pad); y2 = min(rgb.shape[0]-1, y2 + pad)
    crop = rgb[y1:y2+1, x1:x2+1]
    return _enc_png(crop, level=3), crop

@torch.no_grad()
def adaptive_roi_bench(model, img_dir=RAW_IMG_DIR, max_images=120, threshold=BEST_T, save_preview=SAVE_FEW_SAMPLES):
    model.eval()
    paths = sorted(glob.glob(str(img_dir / "*.tif")))
    if len(paths) == 0:
        print("⚠️ No .tif images found"); return None

    # aggregates
    agg = {
        "baseline_jpg90": [], "baseline_webp80": [],
        "chosen_bytes": [], "mode": [], "roi_coverage": [], "edge_bg": [], "conf": []
    }
    saved = 0

    for p in tqdm(paths[:max_images], desc="ROI-Compress (adaptive)"):
        im_bgr = cv2.imread(p); 
        if im_bgr is None: continue
        im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
        H, W = im_rgb.shape[:2]

        # predict mask at 512, back to original
        im_resz = cv2.resize(im_rgb, (512,512))
        t = torch.from_numpy((im_resz/255.0).transpose(2,0,1)).unsqueeze(0).float().to(device)
        prob = torch.sigmoid(model(t)).cpu().numpy()[0,0]
        mask_small = (prob >= threshold).astype(np.uint8)
        mask = cv2.resize(mask_small, (W, H), interpolation=cv2.INTER_NEAREST)

        # metrics for policy
        roi_cov = float(mask.mean())  # 0..1
        conf = float(prob[mask_small==1].mean()) if mask_small.any() else 0.0
        bg_gray = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2GRAY)
        bg_edge = _edge_density(bg_gray*(1-mask))  # edges outside ROI

        # baselines
        b_jpg = len(_enc_jpg(im_rgb, q=90))
        b_wbp = len(_enc_webp(im_rgb, q=80))

        # try candidates
        s1, prev1 = _binary_keep_png(im_rgb, mask)
        s2, prev2 = _soft_keep_webp(im_rgb, mask, q=60)
        s3, prev3 = _roi_only_png(im_rgb, mask)
        cand = {
            "binary_keep_png": len(s1) if s1 is not None else 1e18,
            "soft_keep_webp":  len(s2) if s2 is not None else 1e18,
            "roi_only_png":    len(s3) if s3 is not None else 1e18,
        }

        # simple adaptive policy
        # tune thresholds: ROI small < 0.12; background smooth < 0.06
        if roi_cov < 0.12 and bg_edge < 0.06:
            # background nearly flat, go binary PNG (huge gains)
            mode = "binary_keep_png"; chosen = cand[mode]
        elif roi_cov < 0.08:
            # extremely small ROI → send only crop
            mode = "roi_only_png"; chosen = cand[mode]
        else:
            # keep context but cheap → soft webp
            # (optionally lower q if conf is high)
            q = 55 if conf > 0.8 else 60
            s2b, prev2b = _soft_keep_webp(im_rgb, mask, q=q)
            chosen = len(s2b); mode = f"soft_keep_webp_q{q}"

        # store
        agg["baseline_jpg90"].append(b_jpg)
        agg["baseline_webp80"].append(b_wbp)
        agg["chosen_bytes"].append(chosen)
        agg["mode"].append(mode)
        agg["roi_coverage"].append(roi_cov)
        agg["edge_bg"].append(bg_edge)
        agg["conf"].append(conf)

        # tiny preview if requested
        if save_preview>0 and saved<save_preview:
            overlay = _overlay(im_rgb, mask)
            # choose preview img based on mode
            prev_img = prev2 if "soft_keep" in mode else (prev1 if "binary" in mode else prev3)
            grid = np.concatenate([im_rgb, overlay, prev_img], axis=1)
            cv2.imwrite(str(SAMPLES_OUT_DIR/f"adaptive_{saved:03d}.png"), cv2.cvtColor(grid, cv2.COLOR_RGB2BGR))
            saved += 1

    # summarize
    agg = {k: np.array(v) if k!="mode" else v for k,v in agg.items()}
    def _summ(base):
        return {
            "mean_orig_kb": float(base.mean()/1024),
            "mean_out_kb":  float(agg["chosen_bytes"].mean()/1024),
            "saving_pct":   float((1 - agg["chosen_bytes"].mean()/base.mean())*100.0)
        }

    sum_jpg = _summ(agg["baseline_jpg90"])
    sum_web = _summ(agg["baseline_webp80"])

    print("\n=== Adaptive Policy Results ===")
    print(f"vs JPEG q=90 | mean orig: {sum_jpg['mean_orig_kb']:.1f} KB | mean out: {sum_jpg['mean_out_kb']:.1f} KB | saving: {sum_jpg['saving_pct']:.1f}%")
    print(f"vs WebP q=80 | mean orig: {sum_web['mean_orig_kb']:.1f} KB | mean out: {sum_web['mean_out_kb']:.1f} KB | saving: {sum_web['saving_pct']:.1f}%")

    # mode distribution + diagnostics
    from collections import Counter
    cnt = Counter(agg["mode"])
    print("Mode distribution:", dict(cnt))
    print(f"ROI coverage mean: {agg['roi_coverage'].mean():.3f} | bg edge density mean: {agg['edge_bg'].mean():.3f} | conf mean: {agg['conf'].mean():.3f}")

    return {"summary_vs_jpeg90": sum_jpg, "summary_vs_webp80": sum_web, "modes": cnt,
            "roi_cov_mean": float(agg["roi_coverage"].mean()),
            "edge_bg_mean": float(agg["edge_bg"].mean()),
            "conf_mean": float(agg['conf'].mean())}

# Run it
adaptive_summary = adaptive_roi_bench(model, max_images=120, threshold=BEST_T, save_preview=0)


In [ ]:
# ============================================
# Visualization: Original | Mask Overlay | Compressed
# ============================================
import cv2, torch, numpy as np, matplotlib.pyplot as plt, random, glob
from pathlib import Path

RAW_IMG_DIR = Path("/kaggle/input/private-data-1/patches/Images")
SAMPLES = 3                     # number of random images to visualize
THRESH = float(adaptive_summary["summary_vs_jpeg90"]["mean_orig_kb"]) if "adaptive_summary" in globals() else 0.55

def overlay_mask(rgb, mask):
    """Return RGB overlay of predicted mask"""
    color = np.zeros_like(rgb)
    color[:, :, 1] = (mask * 255)
    return cv2.addWeighted(rgb, 1.0, color, 0.45, 0)

@torch.no_grad()
def show_visualizations(model, img_dir=RAW_IMG_DIR, n=SAMPLES, threshold=0.55):
    files = sorted(glob.glob(str(img_dir / "*.tif")))
    chosen = random.sample(files, n)
    model.eval()

    plt.figure(figsize=(15, n * 4))
    for i, f in enumerate(chosen):
        im_bgr = cv2.imread(f)
        if im_bgr is None: continue
        im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
        h, w = im_rgb.shape[:2]

        # Predict mask
        inp = cv2.resize(im_rgb, (512, 512))
        t = torch.from_numpy((inp / 255.).transpose(2, 0, 1)).unsqueeze(0).float().to(device)
        prob = torch.sigmoid(model(t)).cpu().numpy()[0, 0]
        mask = cv2.resize((prob >= threshold).astype(np.uint8), (w, h), interpolation=cv2.INTER_NEAREST)

        # compressed version (use soft_keep_webp for visual realism)
        m3 = np.dstack([mask]*3).astype(np.uint8)
        bg = cv2.GaussianBlur(im_rgb, (15,15), 0)
        bg = (bg.astype(np.float32) * 0.35).astype(np.uint8)
        comp = (m3 * im_rgb + (1 - m3) * bg).astype(np.uint8)

        # overlay
        overlay = overlay_mask(im_rgb, mask)

        # plot
        plt.subplot(n, 3, 3*i+1); plt.imshow(im_rgb); plt.axis("off"); plt.title("Original")
        plt.subplot(n, 3, 3*i+2); plt.imshow(overlay); plt.axis("off"); plt.title("Predicted Mask Overlay")
        plt.subplot(n, 3, 3*i+3); plt.imshow(comp); plt.axis("off"); plt.title("Compressed (Soft Keep WebP)")
    plt.tight_layout()
    plt.show()

# Run visualization
show_visualizations(model, threshold=0.55)


In [ ]:
# ===============================================
# Generate pseudo-labels for unlabeled set (raw .tif)
# ===============================================
import torch, cv2, numpy as np
from tqdm import tqdm
from pathlib import Path

UNLABELED_DIR = Path("/kaggle/input/private-data-1/patches/Images")
PSEUDO_DIR = Path("/kaggle/working/pseudo_labels"); PSEUDO_DIR.mkdir(parents=True, exist_ok=True)

@torch.no_grad()
def generate_pseudo_labels(model, img_dir=UNLABELED_DIR, save_dir=PSEUDO_DIR, threshold=0.55):
    model.eval()
    imgs = sorted(img_dir.glob("*.tif"))
    print(f"🌀 Generating pseudo-labels for {len(imgs)} images...")
    for p in tqdm(imgs):
        name = p.stem
        img = cv2.imread(str(p))
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        im_resz = cv2.resize(img, (512,512))
        t = torch.from_numpy((im_resz/255.).transpose(2,0,1)).unsqueeze(0).float().to(device)
        prob = torch.sigmoid(model(t)).cpu().numpy()[0,0]
        np.save(save_dir/f"{name}.npy", prob)
    print(f"✅ Saved pseudo-labels to {save_dir}")

# run
generate_pseudo_labels(model, threshold=0.55)


In [ ]:
# ===============================================
# FIXED: Refine pseudo-labels (shape-safe)
# ===============================================
import cv2, numpy as np
from pathlib import Path
from tqdm import tqdm

UNLABELED_DIR = Path("/kaggle/input/private-data-1/patches/Images")
PSEUDO_DIR    = Path("/kaggle/working/pseudo_labels")             # from generate_pseudo_labels()
REFINE_DIR    = Path("/kaggle/working/pseudo_labels_refined"); REFINE_DIR.mkdir(parents=True, exist_ok=True)

EDGE_WEIGHT      = 0.4   # how strongly edges influence [0..1]
SMOOTH_PROB_GAUS = 3     # Gaussian blur kernel for prob (0/None = off)
SMOOTH_EDGE_GAUS = 3     # Gaussian blur kernel for edge map (0/None = off)
IMP_SHARPEN_GAMMA= 1.2   # >1 boosts confident regions (0=off; 1=no change)

def refine_pseudo_labels(in_dir=PSEUDO_DIR, img_dir=UNLABELED_DIR, out_dir=REFINE_DIR):
    files = sorted(in_dir.glob("*.npy"))
    print(f"🔧 Refining {len(files)} pseudo-labels...")
    bad, ok = 0, 0

    for f in tqdm(files):
        prob = np.load(f).astype(np.float32)              # (Hp, Wp) e.g. 512x512
        Ht, Wt = prob.shape

        img_path = img_dir / f"{f.stem}.tif"
        img = cv2.imread(str(img_path))
        if img is None:
            bad += 1
            continue

        # Build edge map at the SAME SHAPE as prob
        img_resz = cv2.resize(img, (Wt, Ht), interpolation=cv2.INTER_LINEAR)
        gray = cv2.cvtColor(img_resz, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200).astype(np.float32) / 255.0  # (Ht, Wt)

        # Optional smoothing
        if SMOOTH_PROB_GAUS and SMOOTH_PROB_GAUS > 0:
            k = int(SMOOTH_PROB_GAUS) | 1  # make odd
            prob = cv2.GaussianBlur(prob, (k, k), 0)
        if SMOOTH_EDGE_GAUS and SMOOTH_EDGE_GAUS > 0:
            k = int(SMOOTH_EDGE_GAUS) | 1
            edges = cv2.GaussianBlur(edges, (k, k), 0)

        # Optional confidence sharpening (gamma)
        if IMP_SHARPEN_GAMMA and IMP_SHARPEN_GAMMA > 0:
            prob = np.clip(prob, 0, 1) ** (1.0 / float(IMP_SHARPEN_GAMMA))

        # Blend with edges (align boundaries)
        refined = (1 - EDGE_WEIGHT) * prob + EDGE_WEIGHT * edges
        refined = np.clip(refined, 0, 1).astype(np.float32)

        np.save(out_dir / f"{f.stem}.npy", refined)
        ok += 1

    print(f"✅ Refined {ok} files → {out_dir}")
    if bad:
        print(f"⚠️ Skipped {bad} files (missing image).")

# Run it
refine_pseudo_labels()


In [ ]:
# ===================================================
# Retraining using refined pseudo labels (semi-supervised)
# ===================================================
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

class SemiSupervisedDataset(Dataset):
    def __init__(self, img_dir, gt_dir, pseudo_dir):
        self.img_dir = img_dir
        self.gt_dir = gt_dir
        self.pseudo_dir = pseudo_dir
        self.files = sorted(img_dir.glob("*.tif"))

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]; name = path.stem
        img = cv2.imread(str(path)); img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (512,512))
        x = torch.from_numpy(img.transpose(2,0,1)/255.).float()
        label_path = self.gt_dir/f"{name}.tif"
        if label_path.exists():
            lbl = cv2.imread(str(label_path),0); lbl = cv2.resize(lbl,(512,512))
            y = torch.from_numpy((lbl>128).astype(np.float32))
        else:
            pseudo = np.load(self.pseudo_dir/f"{name}.npy")
            y = torch.from_numpy((pseudo>0.5).astype(np.float32))
        return x, y.unsqueeze(0)

dataset = SemiSupervisedDataset(
    Path("/kaggle/input/private-data-1/patches/Images"),
    Path("/kaggle/input/private-data-1/patches/Labels"),
    REFINE_DIR
)
loader = DataLoader(dataset,batch_size=8,shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS_FINE = 10

for ep in range(EPOCHS_FINE):
    model.train(); tot=0
    for x,y in tqdm(loader,desc=f"Fine-tune {ep+1}/{EPOCHS_FINE}"):
        x,y=x.to(device),y.to(device)
        p=torch.sigmoid(model(x))
        loss=F.binary_cross_entropy(p,y)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        tot+=loss.item()
    print(f"Epoch {ep+1}: loss={tot/len(loader):.4f}")
torch.save(model.state_dict(),"/kaggle/working/model_finetuned.pth")
print("✅ Fine-tuning complete.")
